In [14]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Union, Set
from enum import Enum
import pickle
import pandas as pd
import statsmodels.api as sm
import numpy as np

from nst_scraper import nst_on_ice_scraper
from db_utils import update_player_db, check_last_update, append_player_ids, get_player_full_name
from team_utils import get_most_recent_game_id
from game_utils import get_game_boxscore, display_boxscore

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
np.set_printoptions(legacy='1.25')

In [15]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', rate='y')
player_stats_df.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,STL,D,26,510.483333,19.633974,0.12,0.35,0.12,0.24,0.47,17.39,2.70,4.35,0.11,6.93,4.11,1.88,0.24,0.35,0.59,1.65,0.82,0.82,0.00,0.0,0.12,2.94,0.59,1.65,2.82,3.53,0.00,0.00,-
1,Brent Burns,CAR,D,25,411.650000,16.466000,0.15,0.73,0.29,0.44,0.87,33.33,5.54,2.63,0.22,15.30,9.18,4.08,0.29,0.58,1.02,0.58,0.29,0.29,0.00,0.0,0.29,4.66,1.60,0.44,2.33,2.62,0.00,0.00,-
2,Corey Perry,EDM,R,25,254.516667,10.180667,0.94,0.00,0.00,0.00,0.94,50.00,4.95,19.05,0.65,11.08,7.54,7.07,3.54,0.00,0.47,3.77,1.18,0.71,0.47,0.0,1.89,1.65,0.94,1.89,4.24,1.18,0.47,0.47,11.79
3,Alex Ovechkin,WSH,L,18,215.250000,11.958333,2.51,1.95,1.11,0.84,4.46,72.73,9.48,26.47,0.91,19.23,13.66,11.15,4.46,0.00,1.39,1.67,0.84,0.84,0.00,0.0,0.00,1.95,1.11,11.43,2.23,0.84,0.00,0.00,-
4,Evgeni Malkin,PIT,C,27,398.983333,14.777160,0.60,1.50,1.05,0.45,2.11,77.78,5.41,11.11,0.78,10.68,7.07,6.32,3.31,0.00,1.35,1.50,0.75,0.75,0.00,0.0,0.75,4.21,1.05,1.95,3.91,2.71,12.78,17.59,6.33


In [16]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', rate='y', pos='g')
goalie_stats_df.head()

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,6,299.833333,49.972222,27.62,25.61,0.928,2.00,0.43,1.70,4.40,3.60,0.818,0.80,-0.01,6.00,5.00,0.833,1.00,-0.28,15.41,15.41,1.000,0.00,0.50,1.80,3.40,39.61,30.40
1,Jonathan Quick,NYR,7,319.583333,45.654762,31.17,29.66,0.952,1.50,1.24,2.60,8.26,7.89,0.955,0.38,1.10,8.64,7.70,0.891,0.94,0.09,13.52,13.33,0.986,0.19,0.25,2.44,5.82,33.71,23.38
2,James Reimer,"ANA, BUF",4,195.983333,48.995833,27.25,24.80,0.910,2.45,-0.05,2.66,3.37,1.84,0.545,1.53,-0.93,12.55,11.63,0.927,0.92,0.58,11.02,11.02,1.000,0.00,0.36,1.84,5.20,32.03,16.88
3,Semyon Varlamov,NYI,10,499.450000,49.945000,25.35,23.07,0.910,2.28,-0.05,2.36,7.33,6.37,0.869,0.96,0.35,6.01,5.29,0.880,0.72,0.00,9.85,9.25,0.939,0.60,-0.28,1.92,4.44,35.72,20.05
4,Jacob Markstrom,N.J,19,915.400000,48.178947,26.61,24.12,0.906,2.49,-0.15,2.21,5.90,4.59,0.778,1.31,-0.26,7.47,6.69,0.895,0.79,0.11,11.21,10.81,0.965,0.39,-0.03,3.60,4.92,40.54,21.32


In [17]:
class Position(Enum):
    C = 'C'
    L = 'L'
    R = 'R'
    D = 'D'
    G = 'G'
    
    @property
    def category(self) -> str:
        if self in {Position.C, Position.L, Position.R}:
            return 'F'
        elif self == Position.D:
            return 'D'
        elif self == Position.G:
            return 'G'
    
    def __str__(self) -> str:
        return self.value

In [18]:
@dataclass
class Player:
    name: str
    team: str
    position: Position
    player_id: Optional[int] = None

    def __str__(self) -> str:
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self) -> Dict[str, Optional[str]]:
        """
        Converts the Player instance into a dictionary.
        
        Returns:
            Dict[str, Optional[str]]: A dictionary representation of the player.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position.value
        }

In [19]:
@dataclass
class Lineup:
    name: str
    forwards: List[Optional[Player]] = field(default_factory=lambda: [None] * 12)
    defense: List[Optional[Player]] = field(default_factory=lambda: [None] * 6)
    goalies: List[Optional[Player]] = field(default_factory=lambda: [None] * 2)
    
    ALLOWED_FORWARD_CATEGORIES = {'F'}
    ALLOWED_DEFENSE_CATEGORY = 'D'
    ALLOWED_GOALIE_CATEGORY = 'G'
    
    def __post_init__(self):
        self.validate_lineup_size()
    
    def validate_lineup_size(self):
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if total_players > 20:
            raise ValueError(f"Total number of players ({total_players}) exceeds the hard limit of 20.")
    
    def add_player(
        self,
        category: str,
        player: Player,
        slot: int,
        allowed_categories: Union[str, Set[str]],
        max_slots: int
    ):
        """
        Adds a player to the specified category and slot after validating their position category.
        Ensures that the total number of players does not exceed 20.
        """
        if isinstance(allowed_categories, str):
            allowed_categories = {allowed_categories}
        elif isinstance(allowed_categories, set):
            allowed_categories = allowed_categories
        else:
            raise TypeError("allowed_categories must be a string or a set of strings.")
        
        if player.position.category not in allowed_categories:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position.value}' "
                f"to {category}. Allowed categories: {', '.join(allowed_categories)}."
            )
        
        if not 0 <= slot < max_slots:
            raise IndexError(f"{category.capitalize()} slot must be between 0 and {max_slots - 1}.")
        
        current_category = getattr(self, category)
        if current_category[slot]:
            existing_player = current_category[slot].name
            print(f"Warning: Slot {slot + 1} in {category} is already occupied by '{existing_player}'. Overwriting.")
        
        # Check total players before adding
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if current_category[slot] is None and total_players >= 20:
            raise ValueError("Cannot add more players. The lineup has reached the hard limit of 20 players.")
        
        current_category[slot] = player
        setattr(self, category, current_category)
        print(f"Added player '{player.name}' to {category.capitalize()} slot {slot + 1}.")
    
    def add_forward(self, player: Player, slot: int):
        self.add_player(
            category='forwards',
            player=player,
            slot=slot,
            allowed_categories=self.ALLOWED_FORWARD_CATEGORIES,
            max_slots=len(self.forwards)
        )
    
    def add_defense(self, player: Player, slot: int):
        self.add_player(
            category='defense',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_DEFENSE_CATEGORY},
            max_slots=len(self.defense)
        )
    
    def set_goalie(self, player: Player, slot: int):
        self.add_player(
            category='goalies',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_GOALIE_CATEGORY},
            max_slots=len(self.goalies)
        )
    
    def adjust_slots(self, category: str, delta: int):
        """
        Adjusts the number of slots in the specified category by delta.
        Allows ±1 adjustment only.
        
        Args:
            category (str): The category to adjust ('forwards' or 'defense').
            delta (int): The change in number of slots (+1 or -1).
        """
        if category not in {'forwards', 'defense'}:
            raise ValueError("Can only adjust 'forwards' or 'defense' categories.")
        if delta not in {-1, 1}:
            raise ValueError("Delta must be either +1 or -1.")
        
        current_slots = getattr(self, category)
        new_slot_count = len(current_slots) + delta
        
        if category == 'forwards':
            if not (11 <= new_slot_count <= 13):
                raise ValueError("Number of forwards can only vary by ±1 from the default of 12.")
        elif category == 'defense':
            if not (5 <= new_slot_count <= 7):
                raise ValueError("Number of defensemen can only vary by ±1 from the default of 6.")
        
        if delta == 1:
            current_slots.append(None)
        elif delta == -1:
            removed_player = current_slots.pop()
            if removed_player:
                print(f"Removed player '{removed_player.name}' from {category}.")
        
        setattr(self, category, current_slots)
        print(f"Adjusted {category} slots to {len(getattr(self, category))}.")
        self.validate_lineup_size()
    
    def display_lineup(self):
        """
        Prints the current lineup in a structured format.
        """
        print(f"Lineup: {self.name}\n")
        
        for category, title in [('forwards', 'Forwards'), ('defense', 'Defense'), ('goalies', 'Goalies')]:
            print(f"{title}:")
            for idx, player in enumerate(getattr(self, category), start=1):
                player_info = str(player) if player else 'Empty'
                print(f"  Slot {idx}: {player_info}")
            print()
    
    def to_dataframe(self) -> pd.DataFrame:
        """
        Converts the lineup into a pandas DataFrame.
        Conditionally includes player attributes if they are present.
        """
        data = []
        for category, pos in [('forwards', 'f'), ('defense', 'd'), ('goalies', 'g')]:
            for idx, player in enumerate(getattr(self, category), start=1):
                player_dict = {
                    'Position': f"{pos}{idx}",
                    'Player': player.name if player else 'Empty'
                }
                # Conditionally add 'player_id' if it exists
                if player and player.player_id is not None:
                    player_dict['Player ID'] = player.player_id
                data.append(player_dict)
        
        df = pd.DataFrame(data)
        
        # Optionally, remove columns where all values are NaN
        df.dropna(axis=1, how='all', inplace=True)
        
        return df
    
    def to_transposed_dataframe(self) -> pd.DataFrame:
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        and the row contains the corresponding player names.
        Conditionally includes additional player attributes if they are present.
        """
        df = self.to_dataframe()
        
        # Initialize dictionaries to hold player names and optional IDs
        player_data = {}
        id_data = {}
        
        for _, row in df.iterrows():
            pos = row['Position']
            player_name = row['Player']
            player_data[pos] = player_name
            
            # Handle 'Player ID' if it exists
            if 'Player ID' in row and pd.notna(row['Player ID']):
                id_data[f"{pos}_ID"] = row['Player ID']
        
        # Combine player names and IDs into a single dictionary
        transposed_data = {**player_data, **id_data}
        
        # Create the transposed DataFrame with a single row
        transposed_df = pd.DataFrame([transposed_data])
        
        return transposed_df

In [20]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    try:
        # !! Take only first position if multiple are listed, this is a hack to deal with the fact that some players have multiple positions listed
        position = row['position'].split(',')[0].strip()
        position_enum = Position(position)  # Convert abbreviation to Position Enum
    except ValueError:
        print(f"Invalid position '{row['position']}' for player '{row['player']}'. Skipping.")
        continue
    
    player = Player(
        name=row['player'],
        team=row['team'],
        position=position_enum
        # player_id is not set initially
    )
    player_list.append(player)

# Convert player list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in player_list]).head()

,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [21]:
goalie_stats_df.head()

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,6,299.833333,49.972222,27.62,25.61,0.928,2.00,0.43,1.70,4.40,3.60,0.818,0.80,-0.01,6.00,5.00,0.833,1.00,-0.28,15.41,15.41,1.000,0.00,0.50,1.80,3.40,39.61,30.40
1,Jonathan Quick,NYR,7,319.583333,45.654762,31.17,29.66,0.952,1.50,1.24,2.60,8.26,7.89,0.955,0.38,1.10,8.64,7.70,0.891,0.94,0.09,13.52,13.33,0.986,0.19,0.25,2.44,5.82,33.71,23.38
2,James Reimer,"ANA, BUF",4,195.983333,48.995833,27.25,24.80,0.910,2.45,-0.05,2.66,3.37,1.84,0.545,1.53,-0.93,12.55,11.63,0.927,0.92,0.58,11.02,11.02,1.000,0.00,0.36,1.84,5.20,32.03,16.88
3,Semyon Varlamov,NYI,10,499.450000,49.945000,25.35,23.07,0.910,2.28,-0.05,2.36,7.33,6.37,0.869,0.96,0.35,6.01,5.29,0.880,0.72,0.00,9.85,9.25,0.939,0.60,-0.28,1.92,4.44,35.72,20.05
4,Jacob Markstrom,N.J,19,915.400000,48.178947,26.61,24.12,0.906,2.49,-0.15,2.21,5.90,4.59,0.778,1.31,-0.26,7.47,6.69,0.895,0.79,0.11,11.21,10.81,0.965,0.39,-0.03,3.60,4.92,40.54,21.32


In [22]:
# Creating Player instances from the goalie_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=Position.G  # Set position to 'G' for goalies
        # player_id is not set initially
    )
    goalie_list.append(player)

# Convert goalie list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,"ANA, BUF",G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,N.J,G


In [23]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-12-04'

In [24]:
# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [25]:
# Check the last update time of the players database
last_update = check_last_update(db_config)

INFO:db_utils:Database connection established.
INFO:db_utils:Last database update was on: 2024-12-04
INFO:db_utils:Database connection closed.


In [26]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_config, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


No need to update the player database. Last update was on: 2024-12-04


In [27]:
def add_player_to_lineup(lineup: Lineup, player: Player, category: str):
    """
    Adds a player to the lineup in the specified category, handling potential errors.
    
    Args:
        lineup (Lineup): The lineup object.
        player (Player): The player to add.
        category (str): The category ('forwards', 'defense', 'goalies').
    """
    try:
        if category == 'forwards':
            slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
            lineup.add_forward(player, slot)
        elif category == 'defense':
            slot = next(i for i, p in enumerate(lineup.defense) if p is None)
            lineup.add_defense(player, slot)
        elif category == 'goalies':
            slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, slot)
        else:
            print(f"Unknown category '{category}'.")
    except StopIteration:
        print(f"No available slots to add player '{player.name}' in category '{category}'.")
    except ValueError as ve:
        print(ve)
    except IndexError as ie:
        print(ie)

In [28]:
# TODO this function just creates a lineup from the player_list and goalie_list
def create_lineup(team) -> Lineup:
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Added player 'Ryan Reaves' to Forwards slot 1.
Added player 'Max Pacioretty' to Forwards slot 2.
Added player 'John Tavares' to Forwards slot 3.
Skipping player 'Oliver Ekman-Larsson': Cannot add player 'Oliver Ekman-Larsson' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Chris Tanev': Cannot add player 'Chris Tanev' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jani Hakanpää': Cannot add player 'Jani Hakanpää' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed categories: F.
Added player 'Max Domi' to Forwards slot 4.
Added player 'William Nylander' to Forwards slot 5.
Added player 'Mitch Marner' to Forwards slot 6.
Added player 'Steven Lorentz' to Forwards slot 7.
Skipping player 'Philippe Myers': Cannot add playe

In [29]:
def create_flexible_lineup(team: str) -> Lineup:
    """
    Creates a flexible lineup for the specified team, allowing for ±1 forward or defense spot.
    
    Args:
        team (str): The team name to filter players.
    
    Returns:
        Lineup: The configured lineup object.
    """
    lineup = Lineup(f"Flexible Lineup for {team}")
    
    # Example logic to adjust slots based on team strategy
    # Here, we simply allow flexibility; implement specific rules as needed
    allow_extra_forward = True  # Example condition
    allow_extra_defense = False  # Example condition
    
    if allow_extra_forward:
        lineup.adjust_slots('forwards', 1)  # Increase forwards to 13
    elif allow_extra_defense:
        lineup.adjust_slots('defense', 1)  # Increase defense to 7
    
    # Add players to the lineup
    forward_count = 0
    defense_count = 0
    goalie_count = 0
    
    for player in player_list:
        if player.team == team:
            if player.position.category == 'F' and forward_count < len(lineup.forwards):
                try:
                    lineup.add_forward(player, forward_count)
                    forward_count += 1
                except ValueError:
                    continue
            elif player.position.category == 'D' and defense_count < len(lineup.defense):
                try:
                    lineup.add_defense(player, defense_count)
                    defense_count += 1
                except ValueError:
                    continue
    
    for goalie in goalie_list:
        if goalie.team == team and goalie_count < len(lineup.goalies):
            try:
                lineup.set_goalie(goalie, goalie_count)
                goalie_count += 1
            except ValueError:
                continue
    
    return lineup

# Create and display a flexible lineup for 'TOR'
# flexible_lineup = create_flexible_lineup('TOR')
# flexible_lineup.display_lineup()

In [30]:
my_lineup.to_dataframe()

,Position,Player
0,f1,Ryan Reaves
1,f2,Max Pacioretty
2,f3,John Tavares
3,f4,Max Domi
4,f5,William Nylander
5,f6,Mitch Marner
6,f7,Steven Lorentz
7,f8,Auston Matthews
8,f9,Alex Nylander
9,f10,David Kampf


In [31]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,Alex Nylander,David Kampf,Connor Dewar,Pontus Holmberg,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll


In [32]:
def assign_player_ids_to_lineup(transposed_lineup_df, db_config):
    """
    Processes the transposed_lineup_df DataFrame to append player IDs next to each player's name.
    
    This function performs the following steps:
        1. Extracts all unique player names from the lineup DataFrame.
        2. Creates Player instances for each player name.
        3. Uses the append_player_ids function to assign player IDs to each Player object.
        4. Inserts new columns into the DataFrame with the corresponding player IDs.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        db_config (dict): Database configuration with keys: dbname, user, password, host, port.
    
    Returns:
        pd.DataFrame: The updated DataFrame with player IDs added.
    """
    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")

    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()

    # Create Player instances
    player_list = []
    for name in player_names:
        if name != 'Empty':
            player = Player(name=name, team=None, position=None)  # Team and position can be set if available
            player_list.append(player)

    # Append player IDs using the existing function
    append_player_ids(player_list, db_config)

    # Create a mapping from player name to player_id
    name_to_id = {player.name: player.player_id for player in player_list if player.player_id is not None}

    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()

    # Insert player_id columns adjacent to each player name column
    for col in player_columns:
        player_name = updated_df.at[0, col]
        if player_name != 'Empty':
            player_id = name_to_id.get(player_name, None)
            id_column = f"{col}_id"
            updated_df[id_column] = player_id
        else:
            id_column = f"{col}_id"
            updated_df[id_column] = None  # Assign None for empty slots

    return updated_df

In [33]:
# Assuming you have already created `transposed_lineup_df` and `db_config`
updated_lineup_df = assign_player_ids_to_lineup(transposed_lineup_df, db_config)

# Display the updated DataFrame
updated_lineup_df

INFO:db_utils:Database connection established.
INFO:db_utils:Assigned player_id 8471817 to Ryan Reaves.
INFO:db_utils:Assigned player_id 8474157 to Max Pacioretty.
INFO:db_utils:Assigned player_id 8475166 to John Tavares.
INFO:db_utils:Assigned player_id 8477503 to Max Domi.
INFO:db_utils:Assigned player_id 8477939 to William Nylander.
INFO:db_utils:Assigned player_id 8478483 to Mitch Marner.
INFO:db_utils:Assigned player_id 8478904 to Steven Lorentz.
INFO:db_utils:Assigned player_id 8479318 to Auston Matthews.
INFO:db_utils:Assigned player_id 8479423 to Alex Nylander.
INFO:db_utils:Assigned player_id 8480144 to David Kampf.
INFO:db_utils:Assigned player_id 8480980 to Connor Dewar.
INFO:db_utils:Assigned player_id 8480995 to Pontus Holmberg.
INFO:db_utils:Assigned player_id 8475171 to Oliver Ekman-Larsson.
INFO:db_utils:Assigned player_id 8475690 to Chris Tanev.
INFO:db_utils:Assigned player_id 8475825 to Jani Hakanpää.
INFO:db_utils:Assigned player_id 8476853 to Morgan Rielly.
INFO:db

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_id,f2_id,f3_id,f4_id,f5_id,f6_id,f7_id,f8_id,f9_id,f10_id,f11_id,f12_id,d1_id,d2_id,d3_id,d4_id,d5_id,d6_id,g1_id,g2_id
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,Alex Nylander,David Kampf,Connor Dewar,Pontus Holmberg,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,8471817,8474157,8475166,8477503,8477939,8478483,8478904,8479318,8479423,8480144,8480980,8480995,8475171,8475690,8475825,8476853,8476931,8479026,8476932,8479361


In [34]:
# player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head() 

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,STL,D,26,510.483333,19.633974,0.12,0.35,0.12,0.24,0.47,17.39,2.70,4.35,0.11,6.93,4.11,1.88,0.24,0.35,0.59,1.65,0.82,0.82,0.00,0.0,0.12,2.94,0.59,1.65,2.82,3.53,0.00,0.00,-
1,Brent Burns,CAR,D,25,411.650000,16.466000,0.15,0.73,0.29,0.44,0.87,33.33,5.54,2.63,0.22,15.30,9.18,4.08,0.29,0.58,1.02,0.58,0.29,0.29,0.00,0.0,0.29,4.66,1.60,0.44,2.33,2.62,0.00,0.00,-
2,Corey Perry,EDM,R,25,254.516667,10.180667,0.94,0.00,0.00,0.00,0.94,50.00,4.95,19.05,0.65,11.08,7.54,7.07,3.54,0.00,0.47,3.77,1.18,0.71,0.47,0.0,1.89,1.65,0.94,1.89,4.24,1.18,0.47,0.47,11.79
3,Alex Ovechkin,WSH,L,18,215.250000,11.958333,2.51,1.95,1.11,0.84,4.46,72.73,9.48,26.47,0.91,19.23,13.66,11.15,4.46,0.00,1.39,1.67,0.84,0.84,0.00,0.0,0.00,1.95,1.11,11.43,2.23,0.84,0.00,0.00,-
4,Evgeni Malkin,PIT,C,27,398.983333,14.777160,0.60,1.50,1.05,0.45,2.11,77.78,5.41,11.11,0.78,10.68,7.07,6.32,3.31,0.00,1.35,1.50,0.75,0.75,0.00,0.0,0.75,4.21,1.05,1.95,3.91,2.71,12.78,17.59,6.33


In [35]:
def get_skater_stats(lineup: Lineup, player_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets stats for players in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing players
        player_stats_df (pd.DataFrame): DataFrame with player statistics
        
    Returns:
        pd.DataFrame: Player statistics ordered according to lineup positions
    """
    # Create ordered list of players (forwards then defense)
    players = []
    # Add forwards in order
    players.extend([p for p in lineup.forwards if p])
    # Add defense in order 
    players.extend([p for p in lineup.defense if p])
    
    # Create ordered list of player names
    player_names = [player.name for player in players]
    
    # Filter stats and reorder to match lineup order
    stats_df = player_stats_df[player_stats_df['player'].isin(player_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(player_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

# lineup_player_stats = get_skater_stats(my_lineup, player_stats_df)

In [36]:
def get_goalie_stats(lineup: Lineup, goalie_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets stats for goalies in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing goalies
        goalie_stats_df (pd.DataFrame): DataFrame with goalie statistics
        
    Returns:
        pd.DataFrame: Goalie statistics ordered according to lineup positions
    """
    # Create ordered list of goalies
    goalies = [goalie for goalie in lineup.goalies if goalie]
    goalie_names = [goalie.name for goalie in goalies]
    
    # Filter stats and reorder to match lineup order
    stats_df = goalie_stats_df[goalie_stats_df['player'].isin(goalie_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(goalie_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

In [37]:
lineup_goalie_stats = get_goalie_stats(my_lineup, goalie_stats_df)
lineup_goalie_stats

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Anthony Stolarz,TOR,14,640.550000,45.753571,28.85,27.54,0.955,1.31,1.23,2.47,6.09,5.25,0.862,0.84,0.24,7.21,6.74,0.935,0.47,0.39,13.68,13.68,1.000,0.00,0.44,3.56,5.81,39.19,16.93
1,Joseph Woll,TOR,9,441.733333,49.081481,25.67,23.91,0.931,1.77,0.49,2.02,5.03,4.35,0.865,0.68,0.22,6.52,6.25,0.958,0.27,0.51,12.77,11.95,0.936,0.81,-0.40,2.58,4.75,39.10,27.92


In [38]:
yesterday

'2024-12-04'

In [39]:
get_most_recent_game_id('TOR', '2024-11-21')

2024020301

In [40]:
def extract_team_lineup(team: str, reference_date: Optional[str] = None) -> Lineup:
    """
    Extracts the most recent lineup for the specified team based on the latest game data.

    This function performs the following steps:
        1. Determines the reference date (defaults to yesterday if not provided).
        2. Retrieves the most recent game ID for the team using `get_most_recent_game_id`.
        3. Fetches the game boxscore data using `get_game_boxscore`.
        4. Processes the boxscore to obtain skaters and goalies using `display_boxscore`.
        5. Constructs and returns a `Lineup` object populated with the team's players.

    Args:
        team (str): The three-letter team code (e.g., 'TOR').
        reference_date (Optional[str]): The reference date in 'YYYY-MM-DD' format. Defaults to yesterday's date.

    Returns:
        Lineup: A `Lineup` object containing the team's players from the most recent game.

    Raises:
        ValueError: If no recent game is found for the team or if the team is not part of the retrieved game.
    """
    # Step 1: Determine the reference date
    if reference_date is None:
        today_datetime = datetime.now()
        yesterday_datetime = today_datetime - timedelta(days=1, hours=6)  # Adjust for UTC offset if necessary
        reference_date = yesterday_datetime.strftime('%Y-%m-%d')

    # Step 2: Retrieve the most recent game ID for the team
    game_id = get_most_recent_game_id(team, reference_date)
    if game_id is None:
        raise ValueError(f"No recent game found for team '{team}' before {reference_date}.")

    # Print the game_id
    print(f"Game ID: {game_id}")

    # Step 3: Fetch the game boxscore data
    game_data = get_game_boxscore(game_id, clean=False)

    # Step 4: Process the boxscore to obtain skaters and goalies
    away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(game_data)

    # Extract team abbrevs to determine if the team is home or away
    away_team_code = game_data.get('awayTeam', {}).get('abbrev')
    home_team_code = game_data.get('homeTeam', {}).get('abbrev')

    if not away_team_code or not home_team_code:
        raise ValueError("Team abbreviations not found in game data.")

    if team.upper() == away_team_code.upper():
        team_side = 'Away'
        skaters = away_skaters
        goalies = away_goalies
    elif team.upper() == home_team_code.upper():
        team_side = 'Home'
        skaters = home_skaters
        goalies = home_goalies
    else:
        raise ValueError(f"Team '{team}' not found in game ID {game_id}.")

    # Step 5: Construct the Lineup object
    lineup = Lineup(name=f"{team.upper()} Lineup from Game {game_id}")

    # Add Skaters to the Lineup
    for _, skater in skaters.iterrows():
        try:
            position_enum = Position(skater['position'])  # Convert to Position Enum
        except ValueError:
            print(f"Invalid position '{skater['position']}' for player '{skater['name']}'. Skipping.")
            continue

        player = Player(
            player_id=skater['playerId'],
            name=get_player_full_name(skater['playerId'], db_config, suppress_log=True),
            team=team.upper(),
            position=position_enum
        )

        # Add player to the appropriate category in the lineup
        if player.position.category == 'F':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
                lineup.add_forward(player, empty_slot)
            except StopIteration:
                print(f"No available forward slot to add player '{player.name}'.")
        elif player.position.category == 'D':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.defense) if p is None)
                lineup.add_defense(player, empty_slot)
            except StopIteration:
                print(f"No available defense slot to add player '{player.name}'.")
        else:
            print(f"Player '{player.name}' has an unrecognized category '{player.position.category}'. Skipping.")

    # Add Goalies to the Lineup
    for _, goalie in goalies.iterrows():
        player = Player(
            player_id=goalie['playerId'],
            name=get_player_full_name(goalie['playerId'], db_config, suppress_log=True),
            team=team.upper(),
            position=Position.G
        )
        try:
            empty_slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, empty_slot)
        except StopIteration:
            print(f"No available goalie slot to add player '{player.name}'.")

    return lineup

In [41]:
chicago_lineup = extract_team_lineup('CHI', '2024-11-22')

Game ID: 2024020313


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Ryan Donato' to Forwards slot 1.
Added player 'Craig Smith' to Forwards slot 2.
Added player 'Jason Dickinson' to Forwards slot 3.
Added player 'Nick Foligno' to Forwards slot 4.
Added player 'Joey Anderson' to Forwards slot 5.
Added player 'Tyler Bertuzzi' to Forwards slot 6.
Added player 'Taylor Hall' to Forwards slot 7.
Added player 'Lukas Reichel' to Forwards slot 8.


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Patrick Maroon' to Forwards slot 9.
Added player 'Teuvo Teravainen' to Forwards slot 10.
Added player 'Ilya Mikheyev' to Forwards slot 11.
Added player 'Connor Bedard' to Forwards slot 12.
Added player 'Connor Murphy' to Defense slot 1.
Added player 'Alec Martinez' to Defense slot 2.
Added player 'Nolan Allan' to Defense slot 3.
Added player 'Wyatt Kaiser' to Defense slot 4.
Added player 'Alex Vlasic' to Defense slot 5.


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'TJ Brodie' to Defense slot 6.
Added player 'Petr Mrazek' to Goalies slot 1.
Added player 'Arvid Soderblom' to Goalies slot 2.


In [42]:
lineup_skater_stats = get_skater_stats(chicago_lineup, player_stats_df)
lineup_skater_stats

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Donato,CHI,C,24,300.166667,12.506944,1.60,0.20,0.20,0.00,1.80,90.00,10.39,15.38,1.08,17.99,13.99,8.60,4.00,0.60,1.20,1.20,0.60,0.60,0.00,0.00,0.60,1.20,1.20,8.60,4.40,1.00,10.39,11.99,9.28
1,Craig Smith,CHI,C,24,244.883333,10.203472,1.23,0.98,0.98,0.00,2.21,75.00,10.78,11.36,0.68,16.17,13.23,7.35,2.94,0.74,1.96,3.92,0.98,0.74,0.00,0.25,0.74,3.19,1.72,4.41,2.70,3.43,0.49,1.72,5.44
2,Jason Dickinson,CHI,C,26,323.000000,12.423077,0.93,0.37,0.37,0.00,1.30,63.64,3.72,25.00,0.41,11.15,6.13,5.76,2.23,0.19,0.93,2.23,1.11,1.11,0.00,0.00,0.56,1.86,1.67,6.50,5.02,3.34,25.26,27.31,8.93
3,Nick Foligno,CHI,L,26,326.266667,12.548718,0.74,0.55,0.37,0.18,1.29,58.33,4.78,15.38,0.58,8.46,6.80,6.25,3.68,0.18,1.47,1.66,0.55,0.37,0.18,0.00,1.29,1.84,0.74,13.61,3.49,1.66,16.55,15.82,9.40
4,Joey Anderson,CHI,R,12,150.833333,12.569444,0.00,0.40,0.40,0.00,0.40,33.33,4.38,0.00,0.43,7.96,5.17,3.98,1.99,0.40,0.80,0.00,0.00,0.00,0.00,0.00,0.40,1.19,2.39,3.18,6.36,2.78,0.00,0.00,-
5,Tyler Bertuzzi,CHI,L,26,312.083333,12.003205,0.19,0.58,0.58,0.00,0.77,40.00,3.85,5.00,0.57,9.04,7.31,4.42,2.88,0.38,0.19,1.15,0.58,0.58,0.00,0.00,1.15,1.92,1.35,3.65,1.92,2.11,0.00,0.00,-
6,Taylor Hall,CHI,L,25,333.216667,13.328667,0.72,0.54,0.18,0.36,1.26,70.00,7.74,9.30,0.61,15.13,10.26,7.56,2.52,0.72,0.36,1.44,0.72,0.72,0.00,0.00,0.72,2.88,2.34,2.88,5.22,3.42,2.34,4.32,6.33
7,Lukas Reichel,CHI,L,22,227.800000,10.354545,0.79,1.32,1.05,0.26,2.11,88.89,8.17,9.68,0.84,12.38,10.01,7.64,3.95,0.79,0.79,1.05,0.53,0.53,0.00,0.00,0.26,0.79,1.58,1.84,3.16,3.16,11.06,22.12,8.78
8,Patrick Maroon,CHI,L,23,238.600000,10.373913,0.25,1.76,1.26,0.50,2.01,80.00,4.53,5.56,0.56,8.80,7.80,4.27,2.51,1.01,0.50,4.53,1.51,1.01,0.50,0.00,1.01,2.51,0.50,10.31,4.02,0.50,2.51,2.51,12.57
9,Teuvo Teravainen,CHI,C,26,328.750000,12.644231,0.37,0.37,0.18,0.18,0.73,44.44,6.57,5.56,0.34,10.59,8.03,4.20,0.55,0.73,0.73,0.73,0.37,0.37,0.00,0.00,0.37,4.02,0.91,0.37,0.91,2.01,0.91,0.37,13.04


In [43]:
lineup_goalie_stats = get_goalie_stats(chicago_lineup, goalie_stats_df)
lineup_goalie_stats

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance
0,Petr Mrazek,CHI,19,924.666667,48.666667,28.88,26.34,0.912,2.53,0.01,2.63,8.05,6.55,0.815,1.49,-0.06,6.29,5.65,0.897,0.65,0.10,12.72,12.33,0.969,0.39,0.02,2.47,7.07,35.91,19.03
1,Arvid Soderblom,CHI,7,350.483333,50.069048,29.10,26.88,0.924,2.23,0.34,2.49,7.36,6.16,0.837,1.20,0.12,7.36,6.68,0.907,0.68,0.19,12.33,11.98,0.972,0.34,0.06,2.74,5.82,37.35,21.92


In [44]:
def calculate_min_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the average time on ice per game as a percentage of total game time (60 minutes).
    
    Args:
        df (pd.DataFrame): DataFrame containing 'toi' and 'gp' columns
        
    Returns:
        pd.DataFrame: Original DataFrame with new 'min%' column added
    """
    df_copy = df.copy()
    df_copy['min%'] = (df_copy['toi'] / df_copy['gp'] / 300 * 100).round(2)
    return df_copy

# # Apply the function to lineup_player_stats
# lineup_player_stats = calculate_min_percentage(lineup_player_stats)
# lineup_player_stats

In [45]:
def sum_min_percentage(df: pd.DataFrame) -> float:
    """
    Calculates the sum of the 'min%' column in the given DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        
    Returns:
        float: The total sum of the 'min%' values.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    total_min_percentage = df['min%'].sum()
    return total_min_percentage

# Example usage:
# total_min_percentage = sum_min_percentage(lineup_player_stats)
# print(f"Total min%: {total_min_percentage}")

In [46]:
def calculate_adj_min(df: pd.DataFrame, total_min_percentage: float) -> None:
    """
    Calculates the adjusted minimum (adj_min) for each player based on their min% and the total min%.
    
    The formula used is:
        adj_min = (min% / total_min_percentage) * 300
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        total_min_percentage (float): The total sum of the 'min%' column.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
        ValueError: If total_min_percentage is not a positive number.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    if total_min_percentage <= 0:
        raise ValueError("total_min_percentage must be a positive number.")
    
    # Calculate and append the 'adj_min' column
    df['adj_min'] = ((df['min%'] / total_min_percentage) * 300).round(2)

# # Example usage:
# calculate_adj_min(lineup_player_stats, total_min_percentage)
# print(lineup_player_stats[['player', 'min%', 'adj_min']])

In [47]:
player_stats_df.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%
0,Ryan Suter,STL,D,26,510.483333,19.633974,0.12,0.35,0.12,0.24,0.47,17.39,2.70,4.35,0.11,6.93,4.11,1.88,0.24,0.35,0.59,1.65,0.82,0.82,0.00,0.0,0.12,2.94,0.59,1.65,2.82,3.53,0.00,0.00,-
1,Brent Burns,CAR,D,25,411.650000,16.466000,0.15,0.73,0.29,0.44,0.87,33.33,5.54,2.63,0.22,15.30,9.18,4.08,0.29,0.58,1.02,0.58,0.29,0.29,0.00,0.0,0.29,4.66,1.60,0.44,2.33,2.62,0.00,0.00,-
2,Corey Perry,EDM,R,25,254.516667,10.180667,0.94,0.00,0.00,0.00,0.94,50.00,4.95,19.05,0.65,11.08,7.54,7.07,3.54,0.00,0.47,3.77,1.18,0.71,0.47,0.0,1.89,1.65,0.94,1.89,4.24,1.18,0.47,0.47,11.79
3,Alex Ovechkin,WSH,L,18,215.250000,11.958333,2.51,1.95,1.11,0.84,4.46,72.73,9.48,26.47,0.91,19.23,13.66,11.15,4.46,0.00,1.39,1.67,0.84,0.84,0.00,0.0,0.00,1.95,1.11,11.43,2.23,0.84,0.00,0.00,-
4,Evgeni Malkin,PIT,C,27,398.983333,14.777160,0.60,1.50,1.05,0.45,2.11,77.78,5.41,11.11,0.78,10.68,7.07,6.32,3.31,0.00,1.35,1.50,0.75,0.75,0.00,0.0,0.75,4.21,1.05,1.95,3.91,2.71,12.78,17.59,6.33


In [48]:
def calculate_ixg_per_60(df: pd.DataFrame) -> None:
    """
    Calculates the expected goals per 60 minutes (ixg_per_60) for each player.

    The formula used is:
        ixg_per_60 = (ixg / toi) * 60

    Args:
        df (pd.DataFrame): DataFrame containing the 'ixg' and 'toi' columns.
        
    Raises:
        KeyError: If either 'ixg' or 'toi' columns are not present in the DataFrame.
        ValueError: If 'toi' contains zero or negative values.
    """
    # Check if required columns exist
    required_columns = {'ixg', 'toi'}
    missing_columns = required_columns - set(df.columns)
    if missing_columns:
        raise KeyError(f"The DataFrame is missing the following required columns: {', '.join(missing_columns)}")
    
    # Check for non-positive 'toi' values to avoid division by zero or negative scaling
    if (df['toi'] <= 0).any():
        raise ValueError("The 'toi' column contains zero or negative values, which are invalid for calculating ixg_per_60.")
    
    # Calculate ixg_per_60 and append as a new column
    df['ixg_per_60'] = (df['ixg'] / df['toi']) * 60
    
    # Optionally, round the values for better readability
    df['ixg_per_60'] = df['ixg_per_60'].round(2)

# Example usage:
# calculate_ixg_per_60(lineup_player_stats)
# print(lineup_player_stats[['player', 'ixg', 'toi', 'ixg_per_60']])

In [49]:
# Define file paths to load the model and transformer
model_filepath = 'models/polynomial_model_degree_1.pkl'
poly_filepath = 'models/polynomial_features_degree_1.pkl'

# Load the regression model
with open(model_filepath, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
print(f"Model loaded from {model_filepath}")

# Load the PolynomialFeatures transformer
with open(poly_filepath, 'rb') as poly_file:
    loaded_poly = pickle.load(poly_file)
print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

# Function to make predictions using the loaded model and transformer
def predict_gpm(new_ixg60_value, model, poly, x_col='ixg60'):
    """
    Predicts 'gpm' using the loaded model and polynomial transformer.
    
    Parameters:
        new_ixg60_value (float): The new ixg60 value for prediction.
        model (RegressionResults): The loaded regression model.
        poly (PolynomialFeatures): The loaded polynomial features transformer.
        x_col (str): The name of the independent variable column. Defaults to 'ixg60'.
        
    Returns:
        predicted_gpm (float): The predicted gpm value.
    """
    # Prepare the input data
    X_new = np.array([[new_ixg60_value]])
    X_new_poly = poly.transform(X_new)
    X_new_poly_const = sm.add_constant(X_new_poly, has_constant='add')
    
    # Create DataFrame with appropriate column names
    feature_names = ['const'] + poly.get_feature_names_out([x_col]).tolist()
    new_data = pd.DataFrame(X_new_poly_const, columns=feature_names)
    
    # Predict
    predicted_gpm = model.predict(new_data)
    return predicted_gpm.iloc[0]

# Example: Predicting 'gpm' for a new ixg60 value
# new_ixg60_value = 50
# predicted_gpm = predict_gpm(new_ixg60_value, loaded_model, loaded_poly)
# print(f"Predicted GPM for ixg60={new_ixg60_value}: {predicted_gpm:.4f}")

Model loaded from models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from models/polynomial_features_degree_1.pkl


In [50]:
# Function to predict GPM for each player and add it to the DataFrame
def add_gpm_to_lineup(lineup_df, model, poly):
    """
    Adds a 'gpm' column to the lineup_player_stats DataFrame using the predict_gpm function.
    
    Args:
        lineup_df (pd.DataFrame): DataFrame containing 'ixg/60' column.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with 'gpm' column added.
    """
    # Define a helper function to handle potential missing or invalid values
    def safe_predict(ixg_60):
        if pd.isna(ixg_60):
            return np.nan
        try:
            return predict_gpm(ixg_60, model, poly)
        except Exception as e:
            print(f"Error predicting GPM for ixg_60={ixg_60}: {e}")
            return np.nan
    
    # Apply the predict_gpm function to each 'ixg/60' value
    lineup_df['gpm'] = lineup_df['ixg/60'].apply(safe_predict)
    return lineup_df

# # Apply the function to add 'gpm' to your DataFrame
# lineup_player_stats = add_gpm_to_lineup(lineup_player_stats, loaded_model, loaded_poly)

# # Display the updated DataFrame with 'gpm'
# print(lineup_player_stats[['player', 'ixg/60', 'gpm']])

In [51]:
def calculate_x_goals(lineup_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates 'x_goals' by multiplying 'gpm' with 'adj_min' for each player.
    
    Args:
        lineup_stats_df (pd.DataFrame): DataFrame containing 'gpm' and 'adj_min' columns.
        
    Returns:
        pd.DataFrame: DataFrame with the new 'x_goals' column added.
        
    Raises:
        KeyError: If 'gpm' or 'adj_min' columns are not present in the DataFrame.
        TypeError: If 'gpm' or 'adj_min' contain non-numeric data.
    """
    # Check if required columns exist
    required_columns = {'gpm', 'adj_min'}
    missing_columns = required_columns - set(lineup_stats_df.columns)
    if missing_columns:
        raise KeyError(f"The DataFrame is missing the following required columns: {', '.join(missing_columns)}")
    
    # Check if 'gpm' and 'adj_min' are numeric
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['gpm']):
        raise TypeError("'gpm' column must be numeric.")
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['adj_min']):
        raise TypeError("'adj_min' column must be numeric.")
    
    # Calculate 'x_goals'
    lineup_stats_df = lineup_stats_df.copy()
    lineup_stats_df['x_goals'] = lineup_stats_df['gpm'] * lineup_stats_df['adj_min']
    
    return lineup_stats_df

In [52]:
def calculate_xg_against_per_60(goalie_stats_df: pd.DataFrame) -> float:
    """
    Calculate the expected goals against (xg_against) per 60 minutes.

    Args:
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics with 'xg_against' and 'toi' columns.

    Returns:
        float: The league average xg_against per 60 minutes.

    Raises:
        KeyError: If required columns are missing from the DataFrame.
        ValueError: If no valid goalies with non-zero 'toi' are found.
    """
    # Ensure required columns are present
    required_columns = {'xg_against', 'toi'}
    missing_columns = required_columns - set(goalie_stats_df.columns)
    if missing_columns:
        raise KeyError(f"Missing columns in goalie_stats_df: {', '.join(missing_columns)}")

    # Drop rows with missing or zero 'toi' to avoid division errors
    valid_goalies = goalie_stats_df.dropna(subset=['xg_against', 'toi'])
    valid_goalies = valid_goalies[valid_goalies['toi'] > 0]

    if valid_goalies.empty:
        raise ValueError("No valid goalies with non-zero 'toi' found in goalie_stats_df.")

    # Calculate xg against per 60 minutes for each goalie
    valid_goalies['xg_against_per_60'] = (valid_goalies['xg_against'] / valid_goalies['toi']) * 60

    return valid_goalies

In [53]:
def calculate_league_avg_xg_against_per_60(goalie_stats_df: pd.DataFrame) -> float:
    """
    Calculate the league average expected goals against (xg_against) per 60 minutes.

    Args:
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics with 'xg_against' and 'toi' columns.

    Returns:
        float: The league average xg_against per 60 minutes.

    Raises:
        KeyError: If required columns are missing from the DataFrame.
        ValueError: If no valid goalies with non-zero 'toi' are found.
    """
    # Ensure required columns are present
    required_columns = {'xg_against', 'toi'}
    missing_columns = required_columns - set(goalie_stats_df.columns)
    if missing_columns:
        raise KeyError(f"Missing columns in goalie_stats_df: {', '.join(missing_columns)}")

    # Drop rows with missing or zero 'toi' to avoid division errors
    valid_goalies = goalie_stats_df.dropna(subset=['xg_against', 'toi'])
    valid_goalies = valid_goalies[valid_goalies['toi'] > 0]

    if valid_goalies.empty:
        raise ValueError("No valid goalies with non-zero 'toi' found in goalie_stats_df.")

    # Calculate xg against per 60 minutes for each goalie
    valid_goalies['xg_against_per_60'] = (valid_goalies['xg_against'] / valid_goalies['toi']) * 60

    # Calculate the league average
    league_avg_xg_against_per_60 = valid_goalies['xg_against_per_60'].mean()

    return league_avg_xg_against_per_60

In [54]:
def calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60):
    """
    Calculate the expected goals against adjusted percentage.
    """
    lineup_goalie_stats['adj%'] = (lineup_goalie_stats['xg_against/60'] / goalie_avg_xg_against_per_60) * 100
    return lineup_goalie_stats

In [55]:
def process_team_lineup(input_date: str, team: str, db_config: Dict[str, str], model, poly) -> pd.DataFrame:
    """
    Processes the team's lineup for a given date and predicts GPM for each player.
    This function performs the following steps:
        1. Calls `nst_on_ice_scraper` for the input date minus one day.
        2. Extracts the team's lineup using `extract_team_lineup`.
        3. Retrieves skater statistics with `get_skater_stats`.
        4. Calculates `min%` using `calculate_min_percentage`.
        5. Computes `adj_min` using `calculate_adj_min`.
        6. Determines `ixg_per_60` using `calculate_ixg_per_60`.
        7. Predicts `gpm` using the loaded polynomial regression model.
        8. Calculcates x_goals by multiplying gpm by adj_min    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        team (str): The three-letter team code (e.g., 'TOR').
        db_config (Dict[str, str]): Database configuration dictionary.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with calculated metrics and predicted GPM.
    
    Raises:
        ValueError: If any step in the data processing pipeline fails.
    """

    try:
        # Step 1: Calculate the date minus one day
        reference_datetime = datetime.strptime(input_date, '%Y-%m-%d') - timedelta(days=1)
        reference_date_str = reference_datetime.strftime('%Y-%m-%d')
        print(f"Fetching data for reference date: {reference_date_str}")

        # Step 2: Call nst_on_ice_scraper for player and goalie stats
        player_stats_df = nst_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=reference_date_str,
            rate='y'
        )
        goalie_stats_df = nst_on_ice_scraper(
            fromseason=20242025,
            thruseason=20242025,
            startdate='',
            enddate=reference_date_str,
            rate='y',
            pos='g'
        )
        print("Player and goalie statistics fetched successfully.")

        # Step 3: Extract team lineup for the input date and team
        lineup = extract_team_lineup(team, input_date)
        print(f"Lineup extracted for team {team} on {input_date}.")

        # Step 4: Get skater statistics for the lineup
        lineup_skater_stats = get_skater_stats(lineup, player_stats_df)

        # Step 5: Calculate min%
        lineup_skater_stats = calculate_min_percentage(lineup_skater_stats)

        # Step 6: Calculate adj_min
        total_min_percentage = sum_min_percentage(lineup_skater_stats)
        calculate_adj_min(lineup_skater_stats, total_min_percentage)

        # Step 7: Calculate ixg_per_60
        # calculate_ixg_per_60(lineup_skater_stats)

        # Step 8: Predict gpm using the polynomial regression model
        lineup_skater_stats = add_gpm_to_lineup(lineup_skater_stats, model, poly)

        # Step 9: Calculate x_goals
        lineup_skater_stats = calculate_x_goals(lineup_skater_stats)

        # Step 10: Get goalie stats for the lineup
        lineup_goalie_stats = get_goalie_stats(lineup, goalie_stats_df)

        # Step 11: Calculate xg_against_per_60
        # lineup_goalie_stats = calculate_xg_against_per_60(lineup_goalie_stats)
        
        # Step 12: Calculate league average xg_against_per_60
        goalie_avg_xg_against_per_60 = goalie_stats_df['xg_against/60'].mean()

        lineup_goalie_stats = calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60)
        
        return lineup_skater_stats, lineup_goalie_stats

    except Exception as e:
        print(f"An error occurred during processing: {e}")
        raise

In [56]:
def load_models(model_filepath='models/polynomial_model_degree_1.pkl',
               poly_filepath='models/polynomial_features_degree_1.pkl'):
    """
    Loads the regression model and PolynomialFeatures transformer from the specified file paths.

    Args:
        model_filepath (str): Path to the saved regression model pickle file.
        poly_filepath (str): Path to the saved PolynomialFeatures transformer pickle file.

    Returns:
        tuple: A tuple containing the loaded regression model and PolynomialFeatures transformer.

    Raises:
        FileNotFoundError: If either of the specified files does not exist.
        pickle.UnpicklingError: If there is an error unpickling the files.
    """
    try:
        # Load the regression model
        with open(model_filepath, 'rb') as model_file:
            loaded_model = pickle.load(model_file)
        print(f"Model loaded from {model_filepath}")

        # Load the PolynomialFeatures transformer
        with open(poly_filepath, 'rb') as poly_file:
            loaded_poly = pickle.load(poly_file)
        print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

        return loaded_model, loaded_poly

    except FileNotFoundError as fnf_error:
        print(f"Error: {fnf_error}")
        raise
    except pickle.UnpicklingError as pickle_error:
        print(f"Error loading pickle files: {pickle_error}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [57]:
# Load the models using the new function
loaded_model, loaded_poly = load_models()

# Call the function with desired date and team
input_date = '2024-12-02'
team = 'TOR'
lineup_skater_stats, lineup_goalie_stats = process_team_lineup(input_date, team, db_config, loaded_model, loaded_poly)


Model loaded from models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from models/polynomial_features_degree_1.pkl
Fetching data for reference date: 2024-12-01
Player and goalie statistics fetched successfully.
Game ID: 2024020378


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Mitch Marner' to Forwards slot 1.
Added player 'Steven Lorentz' to Forwards slot 2.
Added player 'Matthew Knies' to Forwards slot 3.
Added player 'Connor Dewar' to Forwards slot 4.
Added player 'Pontus Holmberg' to Forwards slot 5.
Added player 'Auston Matthews' to Forwards slot 6.
Added player 'Fraser Minten' to Forwards slot 7.
Added player 'Nikita Grebenkin' to Forwards slot 8.


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'William Nylander' to Forwards slot 9.
Added player 'Nicholas Robertson' to Forwards slot 10.
Added player 'John Tavares' to Forwards slot 11.
Added player 'Alex Nylander' to Forwards slot 12.
Added player 'Simon Benoit' to Defense slot 1.
Added player 'Chris Tanev' to Defense slot 2.
Added player 'Jake McCabe' to Defense slot 3.
Added player 'Conor Timmins' to Defense slot 4.
Added player 'Morgan Rielly' to Defense slot 5.


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Oliver Ekman-Larsson' to Defense slot 6.
Added player 'Anthony Stolarz' to Goalies slot 1.
Added player 'Joseph Woll' to Goalies slot 2.
Lineup extracted for team TOR on 2024-12-02.


In [58]:
lineup_skater_stats.head()

,player,team,position,gp,toi,toi/gp,goals/60,total_assists/60,first_assists/60,second_assists/60,total_points/60,ipp,shots/60,sh%,ixg/60,icf/60,iff/60,iscf/60,ihdcf/60,rush_attempts/60,rebounds_created/60,pim/60,total_penalties/60,minor/60,major/60,misconduct/60,penalties_drawn/60,giveaways/60,takeaways/60,hits/60,hits_taken/60,shots_blocked/60,faceoffs_won/60,faceoffs_lost/60,faceoffs_%,min%,adj_min,gpm,x_goals
0,Mitch Marner,TOR,R,25,350.283333,14.011333,0.51,2.06,1.37,0.69,2.57,83.33,6.00,8.57,0.55,11.65,7.19,6.51,1.71,0.51,1.54,0.34,0.17,0.17,0.0,0.0,0.69,3.77,2.23,2.06,1.71,2.06,0.17,0.51,4.28,4.67,18.13,0.008967,0.162574
1,Steven Lorentz,TOR,C,25,229.950000,9.198000,0.52,0.52,0.00,0.52,1.04,57.14,4.96,10.53,0.75,12.00,8.61,5.22,2.61,0.26,0.78,0.00,0.00,0.00,0.0,0.0,0.78,1.57,1.57,16.70,5.48,4.96,1.30,1.57,11.86,3.07,11.92,0.012200,0.145429
2,Matthew Knies,TOR,L,23,310.566667,13.502899,1.55,0.39,0.19,0.19,1.93,58.82,6.38,24.24,0.83,10.63,8.69,7.92,5.02,0.77,1.55,1.16,0.58,0.58,0.0,0.0,0.77,1.74,0.58,9.27,5.02,1.55,0.00,0.00,-,4.50,17.47,0.013494,0.235736
3,Connor Dewar,TOR,C,9,84.783333,9.420370,0.00,0.00,0.00,0.00,0.00,-,4.25,0.00,0.86,13.45,7.08,7.78,4.25,0.00,2.83,0.00,0.00,0.00,0.0,0.0,0.71,4.95,0.71,13.45,7.08,4.25,9.20,17.69,24.21,3.14,12.19,0.013979,0.170401
4,Pontus Holmberg,TOR,R,23,258.466667,11.237681,0.00,0.93,0.23,0.70,0.93,80.00,3.25,0.00,0.31,8.82,5.34,4.41,1.86,0.23,0.23,2.32,1.16,1.16,0.0,0.0,1.39,2.55,0.93,1.86,7.66,2.55,10.21,9.52,12.02,3.75,14.56,0.005087,0.074070


In [59]:
lineup_goalie_stats

,player,team,gp,toi,toi/gp,shots_against/60,saves/60,sv%,gaa,gsaa/60,xg_against/60,hd_shots_against/60,hd_saves/60,hdsv%,hdgaa,hdgsaa/60,md_shots_against/60,md_saves/60,mdsv%,mdgaa,mdgsaa/60,ld_shots_against/60,ld_saves/60,ldsv%,ldgaa,ldgsaa/60,rush_attempts_against/60,rebound_attempts_against/60,avg._shot_distance,avg._goal_distance,adj%
0,Anthony Stolarz,TOR,14,640.550000,45.753571,28.85,27.54,0.955,1.31,1.23,2.47,6.09,5.25,0.862,0.84,0.24,7.21,6.74,0.935,0.47,0.39,13.68,13.68,1.000,0.00,0.44,3.56,5.81,39.19,16.93,100.759062
1,Joseph Woll,TOR,9,441.733333,49.081481,25.67,23.91,0.931,1.77,0.49,2.02,5.03,4.35,0.865,0.68,0.22,6.52,6.25,0.958,0.27,0.51,12.77,11.95,0.936,0.81,-0.40,2.58,4.75,39.10,27.92,82.402148


In [60]:
def attach_xgoals_as_combined_object(lineup_stats_df: pd.DataFrame) -> dict:
    """
    Returns a dictionary containing the lineup stats DataFrame and the x_goals sum.
    
    Args:
        lineup_stats_df (pd.DataFrame): DataFrame containing player statistics, including 'x_goals'.
        
    Returns:
        dict: A dictionary with keys 'lineup_stats' and 'x_goals_sum'.
    """
    x_goals_sum = lineup_stats_df['x_goals'].sum()
    return {
        'lineup_stats': lineup_stats_df,
        'x_goals_sum': x_goals_sum
    }

# Usage Example:
final_lineup_stats_combined = attach_xgoals_as_combined_object(lineup_skater_stats)
final_lineup_stats_combined['lineup_stats']
print(f"Total x_goals: {final_lineup_stats_combined['x_goals_sum']}")

Total x_goals: 2.411165897834231


In [61]:
from pbp_utils import get_matchup_games

temp_data = get_matchup_games('2024-12-05', '2024-12-05')
pd.DataFrame(list(zip(temp_data['game_ids']['id'], temp_data['game_ids']['date'])), columns=['game_id', 'game_date'])

,game_id,game_date
0,2024020410,2024-12-05
1,2024020411,2024-12-05
2,2024020412,2024-12-05
3,2024020413,2024-12-05
4,2024020414,2024-12-05
5,2024020415,2024-12-05
6,2024020416,2024-12-05
7,2024020417,2024-12-05
8,2024020418,2024-12-05


In [62]:
temp_data

{'next_start_date': '2024-12-12',
 'game_ids': {'id': [2024020410,
   2024020411,
   2024020412,
   2024020413,
   2024020414,
   2024020415,
   2024020416,
   2024020417,
   2024020418],
  'date': ['2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05',
   '2024-12-05']}}

In [63]:
from game_utils import get_game_boxscore

get_game_boxscore(2024020417, clean=True)

{'away_team': 'STL', 'home_team': 'CGY'}

In [64]:
def process_matchups_for_date(
    input_date: str,
    db_config: Dict[str, str],
    model,
    poly
) -> pd.DataFrame:
    """
    Processes all matchup games for a given date by extracting team lineups and their statistics.
    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        db_config (Dict[str, str]): Database configuration dictionary.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
    
    Returns:
        pd.DataFrame: A DataFrame with columns 'date', 'game_id', 'away_team', 'home_team',
                      'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'.
    """
    try:
        # Step 1: Call get_matchup_games with the input date as both arguments
        temp_data = get_matchup_games(input_date, input_date)
        game_ids = temp_data.get('game_ids', {}).get('id', [])
        game_dates = temp_data.get('game_ids', {}).get('date', [])
    
        if not game_ids:
            print(f"No games found for the date {input_date}.")
            return pd.DataFrame(columns=['date', 'game_id', 'away_team', 'home_team',
                                         'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'])
    
        if len(game_ids) != len(game_dates):
            print("Mismatch between number of game IDs and game dates.")
            return pd.DataFrame(columns=['date', 'game_id', 'away_team', 'home_team',
                                         'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'])
    
        results = []
    
        # Step 2: Iterate through each game_id and corresponding game_date
        for game_id, game_date in zip(game_ids, game_dates):
            print(f"\nProcessing Game ID: {game_id} on Date: {game_date}")
    
            # Step 3: Get the cleaned boxscore for the current game
            boxscore = get_game_boxscore(game_id, clean=True)
    
            # Extract away and home team abbreviations
            away_team = boxscore.get('away_team')
            home_team = boxscore.get('home_team')
    
            if not away_team or not home_team:
                print(f"Could not extract teams for Game ID: {game_id}. Skipping.")
                continue
    
            print(f"Away Team: {away_team}, Home Team: {home_team}")
    
            # Step 4: Process lineup for the away team
            print(f"Processing lineup for Away Team: {away_team}")
            away_skater_stats, away_goalie_stats = process_team_lineup(
                game_date,
                away_team,
                db_config,
                model,
                poly
            )
    
            # Step 5: Process lineup for the home team
            print(f"Processing lineup for Home Team: {home_team}")
            home_skater_stats, home_goalie_stats = process_team_lineup(
                game_date,
                home_team,
                db_config,
                model,
                poly
            )
    
            # Step 6: Sum x_goals and extract adj% for away team
            away_x_goals = away_skater_stats['x_goals'].sum()
            away_adj_percentage = away_goalie_stats.iloc[0]['adj%'] if not away_goalie_stats.empty else np.nan
    
            # Step 7: Sum x_goals and extract adj% for home team
            home_x_goals = home_skater_stats['x_goals'].sum()
            home_adj_percentage = home_goalie_stats.iloc[0]['adj%'] if not home_goalie_stats.empty else np.nan
    
            # Step 8: Append the results with the new columns
            results.append({
                'date': game_date,
                'game_id': game_id,
                'away_team': away_team,
                'home_team': home_team,
                'away_x_goals': away_x_goals,
                'away_adj%': away_adj_percentage,
                'home_x_goals': home_x_goals,
                'home_adj%': home_adj_percentage
            })
    
        # Convert results to DataFrame with the specified column order
        results_df = pd.DataFrame(results, columns=[
            'date', 'game_id', 'away_team', 'home_team',
            'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'
        ])
    
        print("\nAll matchups processed successfully.")
        return results_df
    
    except Exception as e:
        print(f"An error occurred while processing matchups for date {input_date}: {e}")
        return pd.DataFrame(columns=[
            'date', 'game_id', 'away_team', 'home_team',
            'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%'
        ])

In [65]:
process_matchups_for_date('2024-12-04', db_config, loaded_model, loaded_poly)


Processing Game ID: 2024020406 on Date: 2024-12-04
Away Team: NSH, Home Team: TOR
Processing lineup for Away Team: NSH
Fetching data for reference date: 2024-12-03
Player and goalie statistics fetched successfully.
Game ID: 2024020384


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Filip Forsberg' to Forwards slot 1.
Added player 'Colton Sissons' to Forwards slot 2.
Added player 'Gustav Nyquist' to Forwards slot 3.
Added player 'Mark Jankowski' to Forwards slot 4.
Added player 'Cole Smith' to Forwards slot 5.
Added player 'Fedor Svechkov' to Forwards slot 6.
Added player 'Zachary L'Heureux' to Forwards slot 7.
Added player 'Luke Evangelista' to Forwards slot 8.
Added player 'Jonathan Marchessault' to Forwards slot 9.


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Tommy Novak' to Forwards slot 10.
Added player 'Ryan O'Reilly' to Forwards slot 11.
Added player 'Steven Stamkos' to Forwards slot 12.
Added player 'Luke Schenn' to Defense slot 1.
Added player 'Nick Blankenburg' to Defense slot 2.
Added player 'Alexandre Carrier' to Defense slot 3.
Added player 'Roman Josi' to Defense slot 4.
Added player 'Brady Skjei' to Defense slot 5.
Added player 'Adam Wilsby' to Defense slot 6.


...

INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Quinton Byfield' to Forwards slot 10.
Added player 'Trevor Lewis' to Forwards slot 11.
Added player 'Samuel Helenius' to Forwards slot 12.
Added player 'Joel Edmundson' to Defense slot 1.
Added player 'Jordan Spence' to Defense slot 2.
Added player 'Jacob Moverare' to Defense slot 3.
Added player 'Mikey Anderson' to Defense slot 4.
Added player 'Vladislav Gavrikov' to Defense slot 5.


INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.
INFO:db_utils:Database connection established.
INFO:db_utils:Database connection closed.


Added player 'Brandt Clarke' to Defense slot 6.
Added player 'Erik Portillo' to Goalies slot 1.
Added player 'David Rittich' to Goalies slot 2.
Lineup extracted for team LAK on 2024-12-04.

All matchups processed successfully.


,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%
0,2024-12-04,2024020406,NSH,TOR,2.254712,93.416297,2.444459,100.759062
1,2024-12-04,2024020407,BOS,CHI,2.394648,85.665600,2.090766,107.285965
2,2024-12-04,2024020408,VGK,ANA,2.426434,108.509759,2.450342,138.696685
3,2024-12-04,2024020409,DAL,LAK,2.490927,96.271817,2.574731,108.509759
